# Model Compression Techniques

[Reference Article: Compressing Large Language Models (LLMs) by Shaw Talebi](https://towardsdatascience.com/compressing-large-language-models-llms-9f406eea5b5e)

# Purpose
The purpose of compression LLMs is to reduce model size without sacificing performance.
# Strageties
There are three main ways to do so and they are independent of each other, meaning that the method can be used togheter to potential yield greater compression while maintaining comparable performace.
1. Quantization  
This means to reduce the precision of the parameters in the model.
- Post-training Quantization means to train the model then quantize, improving end user inference speeds
- Quantization Aware Training means to quantize then train, which may yield better performance on specfic downstream tasks
2. Pruning  
This means to remove parameters or even layers from the model.
- Unstuctured pruning removes individual weights and leads to a greater reduction by requires specialized hardware.
- Structured pruning remove entire sturctures and yields less reduction.
3. Knowledge Distillation  
This means to transfer knowledge from a larger (teacher) model to a smaller (student) model.
- When the teacher is another model, the teacher model produces "soft targets" meaning the values produced are proablistic. The student "learns" by comparing its outputs to that of the the teacher model.
- When we use ground truth values to train the student model, those would be called "hard targets" i.e in the case of binary classification 0 or 1
- Another method is to use an existing LLM to generate Synthetic Data which can be feed into the student model.

To reemphasize, these methods are independent and can be used in conjuction with each other.